In [4]:
import pandas as pd

In [6]:
movies_data = pd.read_csv('movies_metadata.csv',low_memory=False)

In [7]:
from ast import literal_eval
movies_data['genres'] = movies_data['genres'].fillna('[]').apply(literal_eval)
movies_data['genres'] = movies_data['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else[])
movies_data['genres'].head(2)

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
Name: genres, dtype: object

In [8]:
genre_split = movies_data.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
genre_split.head(6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


0    Animation
0       Comedy
0       Family
1    Adventure
1      Fantasy
1       Family
dtype: object

In [15]:
genre_split.name = 'Genre'
md = movies_data.drop('genres', axis=1).join(genre_split)

In [17]:
import numpy as np
movies_data['year'] = pd.to_datetime(movies_data['release_date'], errors='coerce')
movies_data['year'] = movies_data['year'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

#### CONTENT BASED RECOMMENDER(METADATA BASED RECOMMENDER)

In [20]:
links_small = pd.read_csv('links_small.csv', error_bad_lines= False)
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [21]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [22]:
from pandas.api.types import is_numeric_dtype
is_numeric_dtype(md['id'])

False

In [23]:
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
md['id'].head(10)

0      862
0      862
0      862
1     8844
1     8844
1     8844
2    15602
2    15602
3    31357
3    31357
Name: id, dtype: int64

In [24]:
md_new = md[md['id'].isin(links_small)]
md_new.shape

(22491, 24)

In [25]:
md_new_sample = md_new.sample(frac = 0.15,random_state=42)
md_new_sample.shape

(3374, 24)

In [27]:
md_new_sample['tagline'] = md_new_sample['tagline'].fillna('')
md_new_sample['description'] = md_new_sample['overview'] + md_new_sample['tagline']
md_new_sample['description'] = md_new_sample['description'].fillna('')

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [29]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),stop_words='english')
tfidf_matrix = tf.fit_transform(md_new_sample['description'])
tfidf_matrix.shape

(3374, 99501)

In [30]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [31]:
cosine_sim

array([[1.        , 0.00783607, 0.        , ..., 0.        , 0.03788309,
        0.00629117],
       [0.00783607, 1.        , 0.        , ..., 0.        , 0.00528033,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.01328575],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.00930783,
        0.        ],
       [0.03788309, 0.00528033, 0.        , ..., 0.00930783, 1.        ,
        0.        ],
       [0.00629117, 0.        , 0.01328575, ..., 0.        , 0.        ,
        1.        ]])

In [32]:
pd.Series(md_new_sample.index, index=md_new_sample['title'])

title
The Spy Next Door                 14850
Just Go with It                   16818
King Creole                        3483
Nancy Drew                        11865
American Wedding                   6437
                                  ...  
Fallen                             1666
Thirst                            15482
Swimming Pool                      6389
Hillbillys in a Haunted House      3340
Betty Fisher and Other Stories     5440
Length: 3374, dtype: int64

In [33]:
md_new_sample = md_new_sample.reset_index()
titles = md_new_sample['title']
indices = pd.Series(md_new_sample.index, index=md_new_sample['title'])

In [34]:
def recommend(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [35]:
recommend('Avatar').head(10)

3012    Lara Croft Tomb Raider: The Cradle of Life
3299                         Hellraiser: Bloodline
1336                                  The American
3046                               A Grand Day Out
3188                                 House Party 2
26                                       Supernova
368                                         Avalon
433                                       Semi-Pro
1640                     Heaven Knows, Mr. Allison
340                               The Book of Life
Name: title, dtype: object